# Preliminary TCGA DGE analysis
### Primary concerns about this analysis:
1. Low statistical power: healthy condition has few samples
2. Missing data: ovarian cancer cohort has no healthy samples

In [1]:
library(tidyverse)
library(TCGAbiolinks)
library(DESeq2)
library(BiocParallel)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects

## Take advantage of parallelization in DESeq2 functions

In [2]:
n_cores <- detectCores()
BiocParallel::register(MulticoreParam(n_cores))

## Constants

In [3]:
projects <- c("TCGA-UCEC", "TCGA-CESC")
data_root <- "../../../../../mnt/d/TCGA"
project_dirs <- sapply(
    projects, 
    (function(p) paste0(data_root, "/", as.character(Sys.Date()), "-", p, "-", "TCGAbiolinks"))
)
tumor_levels <- c("Primary solid Tumor", "Metastatic")
healthy_levels <- c("Solid Tissue Normal")
tumor_def <- "Tumor"
healthy_def <- "Healthy"
proj_idx <- 2

## Functions

In [4]:
# Consolidate levels and (optionally) filter remaining levels
# Ex:
# - {"Primary solid Tumor", "Metastatic"} -> "Tumor"
# - {"Solid Tissue Normal"} -> "Healthy"
# - {"Recurrent Tumor"} -> (rows removed)
consolidate_levels <- function(d, old_tumor_levels, old_healthy_levels, new_tumor_level, new_healthy_level, drop_remaining = TRUE) {
    tumor_mask <- d$definition %in% old_tumor_levels
    healthy_mask <- d$definition %in% old_healthy_levels
    d$definition[tumor_mask] <- new_tumor_level
    d$definition[healthy_mask] <- new_healthy_level
    
    if (drop_remaining) {
        level_mask <- d$definition %in% c(new_tumor_level, new_healthy_level)
        d <- d[, level_mask]
    }
}


# Filter out rows (genes) which are not expressed in any sample
filter_unexpressed <- function(d) {
    not_expr_mask <- rowSums(DESeq2::counts(d)) == 0
    return(d[!not_expr_mask, ])
}


# Select which rows to keep based on adjusted p-value and absolute log2 fold-change
filter_DGE_res <- function(df, max_padj = 0.05, min_l2fc = log2(2)) {
    padj_mask <- df$padj < max_padj
    l2fc_mask <- abs(df$log2FoldChange) > min_l2fc
    final_mask <- padj_mask & l2fc_mask
    return(df[final_mask, ])
}

## Run Query & download

In [5]:
query <- GDCquery(
    project = projects[proj_idx],
    data.category = "Transcriptome Profiling",
    data.type = "Gene Expression Quantification",
    workflow.type = "HTSeq - Counts"
)

--------------------------------------
o GDCquery: Searching in GDC database
--------------------------------------
Genome of reference: hg38
--------------------------------------------
oo Accessing GDC. This might take a while...
--------------------------------------------
ooo Project: TCGA-CESC
--------------------
oo Filtering results
--------------------
ooo By data.type
ooo By workflow.type
----------------
oo Checking data
----------------
ooo Check if there are duplicated cases
ooo Check if there results for the query
-------------------
o Preparing output
-------------------


In [6]:
GDCdownload(query, method = "api", directory = project_dirs[proj_idx], files.per.chunk = 10)

GDCdownload will download 309 files. A total of 77.948874 MB


Downloading: 2.5 MB       

Downloading: 2.5 MB       

Downloading: 2.5 MB       

Downloading: 2.5 MB     

...

Downloading: 2.5 MB       

Downloading: 2.5 MB       

Downloading: 2.5 MB     

Downloading: 2.5 MB       

Downloading: 2.3 MB       

## Prepare data for analysis

In [7]:
data <- GDCprepare(query, directory = project_dirs[proj_idx])

|====================================================|100%                      Completed after 29 s 


Starting to add information to samples
 => Add clinical information to samples
Add FFPE information. More information at: 
=> https://cancergenome.nih.gov/cancersselected/biospeccriteria 
=> http://gdac.broadinstitute.org/runs/sampleReports/latest/FPPP_FFPE_Cases.html
 => Adding subtype information to samples
cesc subtype information from:doi:10.1038/nature21386
Accessing www.ensembl.org to get gene information
Warning message:
“`select_()` is deprecated as of dplyr 0.7.0.
Please use `select()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Warning message:
“`filter_()` is deprecated as of dplyr 0.7.0.
Please use `filter()` instead.
See vignette('programming') for more help
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Cache found
From the 60483 genes we couldn't map 3990


In [8]:
data

class: RangedSummarizedExperiment 
dim: 56493 309 
metadata(1): data_release
assays(1): HTSeq - Counts
rownames(56493): ENSG00000000003 ENSG00000000005 ... ENSG00000281912
  ENSG00000281920
rowData names(3): ensembl_gene_id external_gene_name
  original_ensembl_gene_id
colnames(309): TCGA-JX-A3Q8-01A-11R-A21T-07
  TCGA-C5-A1BK-01B-11R-A13Y-07 ... TCGA-EK-A2R8-01A-21R-A18M-07
  TCGA-JW-A5VK-01A-11R-A28H-07
colData names(131): sample patient ... subtype_GEXP.APOBEC3H.164668
  subtype_patient

### Only want two levels

Lump "Metastatic" and "Primary Solid Tumor" together, define "Healthy" as non-tumor, and filter out data from other levels

In [9]:
data_consolidated <- consolidate_levels(
    data,
    old_tumor_levels = tumor_levels,
    old_healthy_levels = healthy_levels,
    new_tumor_level = tumor_def,
    new_healthy_level = healthy_def,
    drop_remaining = TRUE
)

In [10]:
unique(data_consolidated$definition)
dim(data_consolidated)

[1] "Tumor"   "Healthy"

[1] 56493   309

In [11]:
sum(data_consolidated$definition == tumor_def)
sum(data_consolidated$definition == healthy_def)

[1] 306

[1] 3

## Prep data for DGE analysis

In [12]:
dds <- DESeqDataSet(data_consolidated, design = ~ definition)

renaming the first element in assays to 'counts'
converting counts to integer mode
Warning message in DESeqDataSet(data_consolidated, design = ~definition):
“some variables in design formula are characters, converting to factors”

In [13]:
dds

class: DESeqDataSet 
dim: 56493 309 
metadata(2): data_release version
assays(1): counts
rownames(56493): ENSG00000000003 ENSG00000000005 ... ENSG00000281912
  ENSG00000281920
rowData names(3): ensembl_gene_id external_gene_name
  original_ensembl_gene_id
colnames(309): TCGA-JX-A3Q8-01A-11R-A21T-07
  TCGA-C5-A1BK-01B-11R-A13Y-07 ... TCGA-EK-A2R8-01A-21R-A18M-07
  TCGA-JW-A5VK-01A-11R-A28H-07
colData names(131): sample patient ... subtype_GEXP.APOBEC3H.164668
  subtype_patient

### Filter out genes which are not expressed in *any* samples
If a gene is not expressed in any sample under either condition, we will exclude it.

In [14]:
dds_filtered <- filter_unexpressed(dds)

In [15]:
dim(dds_filtered)

[1] 53763   309

## Perform DGE analysis

In [16]:
ddsSeq <- DESeq(dds_filtered, parallel = TRUE)

estimating size factors
estimating dispersions
gene-wise dispersion estimates: 16 workers
mean-dispersion relationship
final dispersion estimates, fitting model and testing: 16 workers
-- replacing outliers and refitting for 4519 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)
estimating dispersions
fitting model and testing


In [17]:
ddsSeq

class: DESeqDataSet 
dim: 53763 309 
metadata(2): data_release version
assays(6): counts mu ... replaceCounts replaceCooks
rownames(53763): ENSG00000000003 ENSG00000000005 ... ENSG00000281912
  ENSG00000281920
rowData names(26): ensembl_gene_id external_gene_name ... maxCooks
  replace
colnames(309): TCGA-JX-A3Q8-01A-11R-A21T-07
  TCGA-C5-A1BK-01B-11R-A13Y-07 ... TCGA-EK-A2R8-01A-21R-A18M-07
  TCGA-JW-A5VK-01A-11R-A28H-07
colData names(133): sample patient ... sizeFactor replaceable

In [18]:
resultsNames(ddsSeq)

[1] "Intercept"                   "definition_Tumor_vs_Healthy"

### Get DGE results using *Tumor vs. Healthy* statistics
This means that, for some $gene_a$, $\log_{2}fc = 2$ means $gene_a$ was expressed *4 times as much* in the tumor condition as in the healthy condition.

In [19]:
res <- results(ddsSeq, contrast = c("definition", "Tumor", "Healthy"), pAdjustMethod = "BH", parallel = TRUE)

## Combine results with external gene IDs

In [20]:
res_df <- as_tibble(res, rownames = "geneID")
ddsSeq_row_data_df <- rowData(ddsSeq) %>% as_tibble()

### Make sure gene ordering is preserved between dataframes

In [21]:
all(ddsSeq_row_data_df$ensembl_gene_id == res_df$geneID)

[1] TRUE

### Place all gene IDs in the same dataframe as the results statistics

In [22]:
res_genes_df <- res_df %>% mutate(
        external_gene_name = ddsSeq_row_data_df$external_gene_name,
        original_ensembl_gene_id = ddsSeq_row_data_df$original_ensembl_gene_id,
        ensembl_gene_id = res_df$geneID
    ) %>%
    select(-geneID) %>%
    select(ensembl_gene_id, original_ensembl_gene_id, external_gene_name, everything())

In [23]:
head(res_genes_df)

ensembl_gene_id,original_ensembl_gene_id,external_gene_name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000003,ENSG00000000003.13,TSPAN6,3476.033426,0.3218580,0.4442807,0.7244475,4.687911e-01,6.418229e-01
ENSG00000000005,ENSG00000000005.5,TNMD,2.267135,-3.8835788,1.7394600,-2.2326347,2.557304e-02,9.202309e-02
ENSG00000000419,ENSG00000000419.11,DPM1,2638.597733,0.3850439,0.2700207,1.4259791,1.538744e-01,3.143098e-01
ENSG00000000457,ENSG00000000457.12,SCYL3,910.186204,0.2949984,0.2780415,1.0609868,2.886959e-01,4.730806e-01
ENSG00000000460,ENSG00000000460.15,C1orf112,878.032328,2.4829913,0.3159528,7.8587418,3.880110e-15,5.052102e-13
ENSG00000000938,ENSG00000000938.11,FGR,514.668255,-1.1657661,0.5907400,-1.9733997,4.845004e-02,1.444367e-01


In [24]:
dim(res_genes_df)

[1] 53763     9

## Create DEG list

### Drop rows (genes) with NA values

In [25]:
na_mask <- rowSums(is.na(res_genes_df)) > 0
res_genes_df <- res_genes_df[!na_mask, ]

No NAs left?

In [26]:
sum(rowSums(is.na(res_genes_df))) == 0

[1] TRUE

How many rows left?

In [27]:
nrow(res_genes_df)

[1] 30468

### Apply final filters (adj. $p$-values, $\log_2$ fold-change)

In [33]:
deg1_padj <- 0.05
deg1_l2fc <- log2(1.5)
DEG1_df <- filter_DGE_res(res_genes_df, max_padj = deg1_padj, min_l2fc = deg1_l2fc)

How many rows left?

In [34]:
nrow(DEG1_df)

[1] 6637

## Save filtered DEG lists

In [30]:
DEG1_file <- paste0("DEG1", "_padj_", as.character(deg1_padj), "_l2fc_", as.character(deg1_l2fc), ".tsv")

In [31]:
DEG1_file

[1] "DEG1_padj_0.05_l2fc_0.584962500721156.tsv"

In [32]:
# write_tsv(DEG1_df, path = paste(data_root, "DGE_analysis", DEG1_file, sep="/"))